# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II.

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Creating tables

#### Creating artist_song_by_session table
##### Schema:
<pre>sessionId     => Part of the PK and partition key **Necesary for filter**    
itemInSession => Part of the PK and clustering key  **Necesary for filter**   
artist        => Artist name **requested**  
songTitle     => Song title **requested**  
songLenght    => Song lenght **requested**</pre>
Notes on PK: The combination of sessionId and itemInSession has to be unique

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS artist_song_by_session"
query = query + "(sessionId int, itemInSession int, artist text, songTitle text,songLenght decimal, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

#### Inserting into artist_song_by_session

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_session(sessionId,itemInSession,artist,songTitle,songLenght)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### Testing artist_song_by_session table

In [10]:
query = "select artist,songtitle,songlenght from artist_song_by_session WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.songtitle, row.songlenght)

Faithless Music Matters (Mark Knight Dub) 495.3073


#### Creating song_playlist_session table
##### Schema
<pre>userid        => Distribution Key and part of the primary key **necesary for filter**  
sessionId     => Distribution Key and part of the primary key **necesary for filter**  
itemInSession => Clustering Key and part of the primary key, **necesary for sort**  
artist        => Artist name **requested**  
songTitle     => Song name **requested**  
firstName     => User first name **requested**  
lastName      => User last name **requested**</pre>
Notes on Pk: the combination of user and session has to be unique, the itemInSession has to be included to use it as an order criteria

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userid int, sessionId int, itemInSession int, artist text, songTitle text,firstName text, lastName text, PRIMARY KEY ((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

#### Inserting into song_playlist_session

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session(userid,sessionid,iteminsession,artist,songtitle,firstname,lastname)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

#### Testing song_playlist_session table

In [14]:
query = "select artist,songtitle,firstname,lastname from song_playlist_session WHERE userId=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.songtitle, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Creating songs_listened_by table
##### Schema
<pre>songTitle => Distribution Key and part of the primary key **Necesary for filter**
userId    => Clustering key and part of the primary key **Necesary to make unique the relation song-user**
fisrtName => Clustering Key and part of the primary key **Required**
lastName  => Clustering key and part of the primary key **Required**</pre>

Notes on primary key: To answer the question is not relevant if a user has listened the song more than one time, we want to know the unique list of users that have heard.

In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS songs_listened_by"
query = query + "(songTitle text,userid int,firstName text, lastName text, PRIMARY KEY (songTitle,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

#### Inserting into song_library

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_listened_by(songTitle, userId, firstName, lastName)"
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### Testing song_library

In [17]:
query = "select firstname,lastname from songs_listened_by WHERE songTitle='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Droping the tables before closing out the sessions

In [18]:
query = "drop table artist_song_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songs_listened_by"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()